In [ ]:
import sys
!{sys.executable} -m pip install pyconll

In [4]:
#import pyconll
#from pyconll import *
import pandas as pd
import numpy as np
import re

In [5]:
FPath = "/emoji/"

In [3]:
# i_header = ['i_idx','i_text','dummy','i_UD_tag','i_tag','dummy2','i_DP_id','i_DP','dummy3','dummy4']
# c_header = ['c_idx','c_text','dummy','c_UD_tag','c_tag','dummy2','c_DP_id','c_DP','dummy3','dummy4']
i_header = ['idx','text','error_type','UD_tag','tag','Polarity','DP_id','DP','dummy3','dummy4']
c_header = ['idx','text','error_type','UD_tag','tag','Polarity','DP_id','DP','dummy3','dummy4']
data_header = ['send_id','first_language','age','total_score']
whole_header = ['i_idx', 'i_text', 'i_UD_tag', 'i_tag', 'i_DP_id', 'i_DP', 
                'c_idx', 'c_text', 'c_UD_tag', 'c_tag', 'c_DP_id', 'c_DP', 
                'send_id','first_language', 'age', 'total_score']
whole_df_m = pd.DataFrame(columns = whole_header)

In [6]:
def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [7]:
# Function to insert row in the dataframe 
def Insert_row(row_number, df, row_value): 
    # Starting value of upper half 
    start_upper = 0
    # End value of upper half 
    end_upper = row_number 
    # Start value of lower half 
    start_lower = row_number 
   
    # End value of lower half 
    end_lower = df.shape[0] 
   
    # Create a list of upper_half index 
    upper_half = [*range(start_upper, end_upper, 1)] 
   
    # Create a list of lower_half index 
    lower_half = [*range(start_lower, end_lower, 1)] 
   
    # Increment the value of lower half by 1 
    lower_half = [x.__add__(1) for x in lower_half] 
   
    # Combine the two lists 
    index_ = upper_half + lower_half 
   
    # Update the index of the dataframe 
    df.index = index_ 
   
    # Insert a row at the end 
    df.loc[row_number] = row_value 
    
    # Sort the index labels 
    df = df.sort_index() 
   
    # return the dataframe 
    return df 

In [8]:
def get_token_in_str(s):
    # extract a token from a string
    token = ''
    for char in s:
        if not char.isalpha():
            token = char
    return token

In [9]:
def get_letters(s):
    # extract all letter from a string
    return ''.join(x for x in s if x.isalpha())

In [44]:
cesl_file, esl_file = open(FPath+"en_cesl-ud-dev.conllu",'r'), open(FPath+"en_esl-ud-dev.conllu",'r')
correct_data, orgin_data = '', ''
for idx,line in enumerate(cesl_file.readlines()):
    correct_data += line + '\n'
entire_data_list = correct_data.split('\n\n\n\n')
del entire_data_list[-1]

for idx,line in enumerate(esl_file.readlines()):
    orgin_data += line + '\n'
entire_org_list = orgin_data.split('\n\n\n\n')
del entire_org_list[-1]

In [45]:
learner_data_mapping = open(FPath+"corrected_",'r')
send_id,send_id_uni,age,fst_language,total_score = [],[],[],[],[]
for idx,line in enumerate(learner_data_mapping.readlines()):
    if '# send_id' in line:
        tmp_send_id = line.split('= ')[1].split('-',2)
        send_id.append(tmp_send_id[0] + '-' + tmp_send_id[1])
    if '# First language' in line:
        fst_language.append(line.split('= ')[1].strip('\n'))
    if '# Age' in line:
        age.append(line.split('= ')[1].strip('\n'))
    if '# Total score' in line:  
        total_score.append(line.split('= ')[1].strip('\n'))
learner_data_df = pd.DataFrame({'send_id': send_id,
                                'first_language': fst_language,
                                'age': age,
                                'total_score': total_score
                               }).drop_duplicates(subset = None, 
                                                  keep = 'first', 
                                                  inplace = False).reset_index(drop=True)

In [52]:
learner_data_df

,send_id,first_language,age,total_score
0,0102_2000_6-doc1884.xml,French,31-40,17.0
1,0102_2000_12-doc624.xml,Russian,16-20,25.00
2,0102_2000_12-doc1562.xml,Turkish,16-20,32.00
3,0102_2000_12-doc180.xml,German,Missing,25.00
4,0102_2000_12-doc2135.xml,Turkish,<16,38.00
5,0102_2000_12-doc2121.xml,Japanese,26-30,22.00
6,0102_2000_12-doc578.xml,Spanish,26-30,29.00
7,0102_2000_12-doc1549.xml,Polish,21-25,25.00
8,0102_2000_12-doc342.xml,Russian,21-25,22.00
9,0102_2000_12-doc144.xml,German,21-25,34.00


In [47]:
split_tmp_s = [s.split('\t') for s in entire_data_list[0].split('\n\n')]
split_tmp_s_org = [s.split('\t') for s in entire_org_list[0].split('\n\n')]

In [48]:
# get error string, correct word and different error types
err_str = ['<ns'+s.partition('<ns')[2] 
           for s in re.findall("<ns(?!type.*.*).*</ns>",
                               split_tmp_s[2][0])[0].split('</ns>')
           if '<ns' in s]

err_type = [find_between_r(s, '<ns type="', '">') 
            for s in err_str]

err_word = [find_between_r(s, '<i>', '</i>')
            for s in err_str]

cor_word = [find_between_r(s, '<c>', '</c>')
            for s in err_str]

In [49]:
err_str,err_type,err_word,cor_word,split_tmp_s_org[2]

(['<ns type="UD"><i>the</i>', '<ns type="S"><i>anyting</i><c>anything</c>'],
 ['UD', 'S'],
 ['the', 'anyting'],
 ['', 'anything'],
 ['# error_annotation = There is a great opportunity to go there because <ns type="UD"><i>the</i></ns> students need not pay <ns type="S"><i>anyting</i><c>anything</c></ns>.'])

In [12]:
# insert row to the correct dataframe, it doesnt conside the error type of missing or 
def insert_to_df_no_punc(compared_df, be_inserted_df, check_wl, err_type, err_idx, header):
    """
    check_wl: check word list, if missing word, then insert error into correct dataframe, if word is unnecessary
    then insert correct word to error dataframe
    """
    
    for idx, check_row in compared_df.iterrows():
        if be_inserted_df.loc[idx].equals(check_row):
            # if the row are the same, then don't need to modify
            continue
        elif be_inserted_df.loc[idx]['idx'] == '_':
            # check if the inserted columns
            continue
        elif check_row['text'] == check_wl[err_idx]:
            compared_df.loc[idx, 'error_type'] = err_type[err_idx]
            tmp_row = [['_', check_wl[err_idx],
                        err_type[err_idx],'_',
                        '_','_','_','_','_','_']]
            tmp_df = pd.DataFrame(tmp_row,
                                  columns = header)

            be_inserted_df = Insert_row(idx, 
                                        be_inserted_df, 
                                        tmp_df.loc[0])

            break
    return compared_df,be_inserted_df

if unnecessary, then insert row in the correct dataframe, 
elif missing, then insert row in the original dataframe

In [50]:
df_c = pd.DataFrame(split_tmp_s[3:],
                    columns = c_header)
df_i = pd.DataFrame(split_tmp_s_org[3:],
                    columns = i_header)
err = err_str[0]
for err_idx in range(0,2): #need to be changed
    if 'type="U' in err_str[err_idx]:
        df_i, df_c = insert_to_df_no_punc(df_i, df_c, err_word, err_type, err_idx, c_header)
    elif 'type="M' in err:
        if 'type="MP"' not in err_str[err_idx]:
            df_c, df_i = insert_to_df_no_punc(df_c, df_i, cor_word, err_type, err_idx, i_header)   
        else:
            if len(cor_word[err_idx].split(' ')) == len(err_word[err_idx].split(' ')):
                df_c, df_i = insert_to_df_no_punc(df_c, df_i, cor_word, err_type, err_idx, i_header)

In [51]:
df_i

,idx,text,error_type,UD_tag,tag,Polarity,DP_id,DP,dummy3,dummy4
0,1,There,_,PRON,EX,_,2,expl,_,_
1,2,is,_,VERB,VBZ,_,0,root,_,_
2,3,a,_,DET,DT,_,5,det,_,_
3,4,great,_,ADJ,JJ,_,5,amod,_,_
4,5,opportunity,_,NOUN,NN,_,2,nsubj,_,_
5,6,to,_,PART,TO,_,7,mark,_,_
6,7,go,_,VERB,VB,_,5,acl,_,_
7,8,there,_,ADV,RB,_,7,advmod,_,_
8,9,because,_,SCONJ,IN,_,12,mark,_,_
9,10,the,UD,DET,DT,_,11,det,_,_


In [14]:
err_str,err_type,err_word,cor_word,split_tmp_s_org[2]

(['<ns type="MP"><c>,</c>',
  '<ns type="UD"><i>the</i>',
  '<ns type="FV"><i>smoking</i><c>smoke</c>',
  '<ns type="MP"><i>no smoking</i><c>no-smoking</c>'],
 ['MP', 'UD', 'FV', 'MP'],
 ['', 'the', 'smoking', 'no smoking'],
 [',', '', 'smoke', 'no-smoking'],
 ['# error_annotation = Firstly <ns type="MP"><c>,</c></ns> in <ns type="UD"><i>the</i></ns> school it is not permitted to <ns type="FV"><i>smoking</i><c>smoke</c></ns> in the <ns type="MP"><i>no smoking</i><c>no-smoking</c></ns> area or in the classroom.'])

In [15]:
df_c = pd.DataFrame(split_tmp_s[3:],
                    columns = c_header)
df_i = pd.DataFrame(split_tmp_s_org[3:],
                    columns = i_header)
err_idx = 3
for idx, df_c_row in df_c.iterrows():
    # check the dataframe line by line
    if df_i.loc[idx].equals(df_c_row):
            continue
    elif df_c_row['text'] == cor_word[err_idx]:
        df_c.loc[idx, 'error_type'] = err_type[err_idx]
        tmp_row = [['_', cor_word[err_idx],
                    err_type[err_idx],'_',
                    '_','_','_','_','_','_']]
        tmp_df = pd.DataFrame(tmp_row,
                              columns = i_header)
        df_i =Insert_row(idx, df_i, tmp_df.loc[0])
#         df_i = Insert_row(idx, df_i, tmp_df.loc[0])
#         print('true')
#         break

In [41]:
token = get_token_in_str('no-smoking')

token

'-'

In [ ]:
select_idx, i_sen_with_u, i_sen_with_m, i_sen_with_m_u,i_others,i_no_need = [],[],[],[],[],[]
select_idx, sen_with_u, sen_with_m, sen_with_m_u,others,no_need = [],[],[],[],[],[]
for idx, tmp_s in enumerate(entire_data_list):
    split_tmp_s, split_tmp_s_org = tmp_s.split('\n\n'), entire_org_list[idx].split('\n\n')
    if len(split_tmp_s) != len(split_tmp_s_org) and split_tmp_s[0]==split_tmp_s_org[0]:
        if '<ns type="U' in tmp_s and '<ns type="M' in tmp_s:
            sen_with_m_u.append(tmp_s)
            i_sen_with_m_u.append(entire_org_list[idx])
            continue
        elif '<ns type="U' in tmp_s:
            sen_with_u.append(tmp_s)
            i_sen_with_u.append(entire_org_list[idx])
            continue
        elif '<ns type="M' in tmp_s:
            sen_with_m.append(tmp_s)
            i_sen_with_m.append(entire_org_list[idx])
            continue
        else:
            others.append(tmp_s)
            i_others.append(entire_org_list[idx])
    else:
        no_need.append(tmp_s)
        i_no_need.append(entire_org_list[idx])

In [ ]:
def refract_df_with_one_row(idx,i_test_df,test_df,check_word_list):
    mark_idx = i_test_df.loc[idx]['i_idx']-1
    mark_dep_id = i_test_df.loc[idx]['i_DP_id']
    increment_num = len(check_word_list)
    for num in i_test_df.loc[(i_test_df['i_DP_id'] >= mark_idx + 1)]['i_DP_id'].index:
        i_test_df.loc[num, 'i_DP_id'] = i_test_df.loc[num, 'i_DP_id'] + 1
    for num in range(mark_idx,len(i_test_df)):
        i_test_df.loc[num, 'i_idx'] = i_test_df.loc[num, 'i_idx'] + 1
    for num in range(len(check_word_list)):
        i_test_df.columns = np.arange(0,len(i_test_df.columns))
        test_df.columns = np.arange(0,len(test_df.columns))
        i_test_df = Insert_row(mark_idx+num, i_test_df, test_df.loc[mark_idx+num])
        i_test_df.columns = i_header
        test_df.columns = c_header
    return i_test_df

In [ ]:
for idx,sentence_m in enumerate(sen_with_m):
    test = [s.split('\t') for s in sentence_m.split('\n\n')]
    test_df = pd.DataFrame(test[3:],columns=c_header)
    test_df['c_idx'],test_df['c_DP_id'] = test_df['c_idx'].apply(int), test_df['c_DP_id'].apply(int)
    
    i_test = [s.split('\t') for s in i_sen_with_m[idx].split('\n\n')]
    i_test_df = pd.DataFrame(i_test[3:],columns=i_header)
    i_test_df['i_idx'],i_test_df['i_DP_id']=i_test_df['i_idx'].apply(int),i_test_df['i_DP_id'].apply(int)
    
    current_send_id = (test[0][0].split('= ')[1].split('-',2)[0] 
                       + '-' 
                       + test[0][0].split('= ')[1].split('-',2)[1])
    cur_learner_infos = learner_data_df.loc[learner_data_df['send_id'] == current_send_id].reset_index()
    first_language, age, total_score = (cur_learner_infos['first_language'][0],
                                        cur_learner_infos['age'][0],
                                        cur_learner_infos['total_score'][0])
    find_err_list= sentence_m.split('\n\n')[2].split('<ns type="M')
    for sen in find_err_list[1:]:
        (start,end)=sen.find('<c>',1), sen.find('</c></ns>',1)
        check_word_list = sen[start+len('<c>'):end].split()
        print(sen,check_word_list)
        for row in i_test_df.iterrows():
            idx = int(row[1]['i_idx'])-1
            if row[1]['i_text'] == test_df.loc[idx]['c_text']:
                continue
            elif test_df.loc[idx]['c_text'] == check_word_list[0]:
                i_test_df = refract_df_with_one_row(idx,i_test_df,test_df,check_word_list)
                break

    cur_learner_df = pd.DataFrame(list(zip([current_send_id]*len(test_df),
                                           [first_language]*len(test_df),
                                           [age]*len(test_df),
                                           [total_score]*len(test_df))),
                                 columns = data_header)
    tpm_entire_df = pd.concat([i_test_df[['i_idx','i_text','i_UD_tag',
                                          'i_tag','i_DP_id','i_DP']],
                               test_df[['c_idx','c_text','c_UD_tag',
                                          'c_tag','c_DP_id','c_DP']],
                               cur_learner_df],
                              axis=1,
                              join='outer')
    whole_df_m.append(tpm_entire_df)


In [ ]:
(start,end)=sen.find('<c>',1), sen.find('</c></ns>',1)
check_word_list = sen[start+len('<c>'):end].split()

In [ ]:
tpm_entire_df

In [ ]:
current_send_id = (test[0][0].split('= ')[1].split('-',2)[0] 
                   + '-' 
                   + test[0][0].split('= ')[1].split('-',2)[1])
cur_learner_infos = learner_data_df.loc[learner_data_df['send_id'] == current_send_id].reset_index()
first_language, age, total_score = (cur_learner_infos['first_language'][0],
                                    cur_learner_infos['age'][0],
                                    cur_learner_infos['total_score'][0])

In [ ]:
find_err_list= sen_with_m[0].split('\n\n')[2].split('<ns type="M')
for sen in find_err_list[1:]:
    (start,end)=sen.find('<c>',1), sen.find('</c></ns>',1)
    check_word_list = sen[start+len('<c>'):9].split()
    print(sen,check_word_list)
    for row in i_test_df.iterrows():
        idx = int(row[1]['i_idx'])-1
        if row[1]['i_text'] == test_df.loc[idx]['c_text']:
            continue
        elif test_df.loc[idx]['c_text'] == check_word_list[0]:
            i_test_df = refract_df_with_one_row(idx,i_test_df,test_df,check_word_list)
            break
            
cur_learner_df = pd.DataFrame(list(zip([current_send_id]*len(test_df),
                                       [first_language]*len(test_df),
                                       [age]*len(test_df),
                                       [total_score]*len(test_df))),
                             columns = data_header)
tpm_entire_df = pd.concat([i_test_df[['i_idx','i_text','i_UD_tag',
                                      'i_tag','i_DP_id','i_DP']],
                           test_df[['c_idx','c_text','c_UD_tag',
                                      'c_tag','c_DP_id','c_DP']],
                           cur_learner_df],
                          axis=1,
                          join='outer')

In [75]:
cesl_train = open("en_cesl-ud-train.conllu",'r')
send_id,send_id_uni= [],[]
correct_train = ''
for idx,line in enumerate(cesl_train.readlines()):
    correct_train += line + '\n'
    if 'sent_id' in line:
        print(line)
        tmp_send_id = line.split('= ')[1].split('-',2)
        send_id.append(tmp_send_id[0] + '-' + tmp_send_id[1])
        send_id_uni.append(tmp_send_id[0] + '-' + tmp_send_id[1] + '-' + tmp_send_id[2].strip('\n'))
entire_data_train = correct_train.split('\n\n\n\n')
del entire_data_train[-1]

# sent_id = 0100_2000_6-doc2664.xml-34

# sent_id = 0102_2000_6-doc648.xml-6

# sent_id = 0102_2000_6-doc1081.xml-16

# sent_id = 0102_2000_12-doc724.xml-11

# sent_id = 0100_2000_12-doc567.xml-13

# sent_id = 0102_2000_6-doc1098.xml-22

# sent_id = 0100_2000_6-doc166.xml-14

# sent_id = 0100_2000_12-doc2263.xml-20

# sent_id = 0100_2000_6-doc395.xml-38

# sent_id = 0102_2000_6-doc397.xml-15

# sent_id = 0102_2000_6-doc2093.xml-16

# sent_id = 0100_2000_6-doc57.xml-19

# sent_id = 0102_2000_12-doc472.xml-8

# sent_id = 0100_2001_6-doc3235.xml-18

# sent_id = 0100_2000_6-doc705.xml-29

# sent_id = 0100_2000_6-doc15.xml-1

# sent_id = 0100_2000_6-doc2084.xml-3

# sent_id = 0102_2000_12-doc724.xml-10

# sent_id = 0102_2000_6-doc1731.xml-17

# sent_id = 0102_2000_6-doc2203.xml-18a

# sent_id = 0102_2000_6-doc2203.xml-18b

# sent_id = 0100_2000_6-doc1877.xml-19

# sent_id = 0100_2000_12-doc217.xml-21

# sent_id = 0102_2000_12-doc107.xml-6

# sent_id = 0100_2000_6-doc2574.xml-17

# sent_id =


# sent_id = 0100_2000_6-doc1784.xml-32

# sent_id = 0102_2000_6-doc1425.xml-5

# sent_id = 0100_2001_6-doc2985.xml-18

# sent_id = 0100_2000_6-doc2.xml-44

# sent_id = 0102_2000_6-doc622.xml-2

# sent_id = 0102_2000_6-doc614.xml-7

# sent_id = 0102_2000_6-doc1541.xml-6

# sent_id = 0102_2000_6-doc2402.xml-12

# sent_id = 0102_2000_6-doc2695.xml-18

# sent_id = 0102_2000_6-doc2333.xml-3

# sent_id = 0100_2001_6-doc3039.xml-22

# sent_id = 0102_2000_6-doc2762.xml-16

# sent_id = 0100_2000_6-doc862.xml-24

# sent_id = 0100_2000_6-doc2.xml-37

# sent_id = 0100_2000_6-doc470.xml-23

# sent_id = 0100_2000_6-doc346.xml-3

# sent_id = 0100_2000_6-doc85.xml-16

# sent_id = 0100_2000_12-doc2599.xml-5

# sent_id = 0100_2000_6-doc358.xml-21

# sent_id = 0102_2000_6-doc2609.xml-5

# sent_id = 0100_2000_6-doc58.xml-23

# sent_id = 0102_2000_12-doc2419.xml-19

# sent_id = 0102_2000_6-doc1851.xml-15

# sent_id = 0102_2000_6-doc2107.xml-5

# sent_id = 0102_2000_6-doc1241.xml-5

# sent_id = 0100_2000_1

# sent_id = 0102_2000_6-doc1032.xml-1

# sent_id = 0100_2000_12-doc2394.xml-20

# sent_id = 0102_2001_12-doc2894.xml-27

# sent_id = 0100_2000_6-doc148.xml-6

# sent_id = 0102_2000_12-doc458.xml-22

# sent_id = 0102_2000_12-doc1562.xml-3

# sent_id = 0100_2000_6-doc325.xml-18

# sent_id = 0100_2000_12-doc563.xml-12

# sent_id = 0102_2000_6-doc2498.xml-5

# sent_id = 0100_2001_6-doc3121.xml-3

# sent_id = 0102_2001_3-doc3011.xml-24

# sent_id = 0100_2000_12-doc964.xml-23

# sent_id = 0102_2000_6-doc1056.xml-30

# sent_id = 0102_2000_6-doc513.xml-3

# sent_id = 0102_2000_6-doc1261.xml-21

# sent_id = 0102_2000_12-doc2251.xml-22

# sent_id = 0102_2000_6-doc287.xml-13

# sent_id = 0102_2000_6-doc2429.xml-28a

# sent_id = 0102_2000_6-doc2429.xml-28b

# sent_id = 0100_2000_6-doc84.xml-12

# sent_id = 0102_2000_12-doc124.xml-25

# sent_id = 0100_2000_12-doc2130.xml-12

# sent_id = 0100_2000_12-doc1415.xml-4

# sent_id = 0102_2000_6-doc1893.xml-25

# sent_id = 0100_2000_12-doc1595.xml-24

# se

In [22]:
err_train = list(set([get_letters(i[:2]) for i in correct_train.split('<ns type="')]))[1:]

In [43]:
len(send_id_uni)

4124

In [55]:
# fill the error types
entire_data_train[0].split('<ns type="')[1:]

['S"><i>shoked</i><c>shocked</c></ns> because I had ',
 'S"><i>alredy</i><c>already</c></ns> spoken with them and I had ',
 'RV"><i>taken</i><c>got</c></ns> two autographs.\n\n1\tI\t_\tPRON\tPRP\t_\t3\tnsubj\t_\t_\n\n2\twas\t_\tAUX\tVBD\t_\t3\tcop\t_\t_\n\n3\tshocked\t_\tADJ\tJJ\t_\t0\troot\t_\t_\n\n4\tbecause\t_\tSCONJ\tIN\t_\t8\tmark\t_\t_\n\n5\tI\t_\tPRON\tPRP\t_\t8\tnsubj\t_\t_\n\n6\thad\t_\tAUX\tVBD\t_\t8\taux\t_\t_\n\n7\talready\t_\tADV\tRB\t_\t8\tadvmod\t_\t_\n\n8\tspoken\t_\tVERB\tVBN\t_\t3\tadvcl\t_\t_\n\n9\twith\t_\tADP\tIN\t_\t10\tcase\t_\t_\n\n10\tthem\t_\tPRON\tPRP\t_\t8\tobl\t_\t_\n\n11\tand\t_\tCCONJ\tCC\t_\t14\tcc\t_\t_\n\n12\tI\t_\tPRON\tPRP\t_\t14\tnsubj\t_\t_\n\n13\thad\t_\tAUX\tVBD\t_\t14\taux\t_\t_\n\n14\tgot\t_\tVERB\tVB\t_\t8\tconj\t_\t_\n\n15\ttwo\t_\tNUM\tCD\t_\t16\tnummod\t_\t_\n\n16\tautographs\t_\tNOUN\tNNS\t_\t14\tobj\t_\t_\n\n17\t.\t_\tPUNCT\t.\t_\t3\tpunct\t_\t_']

In [89]:
df_error_types = pd.DataFrame({'send_id': send_id,
             'unique_send_id': send_id_uni}).set_index(['unique_send_id'])
df_error_types = df_error_types.reindex(columns = ['send_id'] 
                       + err_train).fillna(0)    
for uni_conllu in entire_data_train:
    tmp_send_id = uni_conllu.split('\n\n')[0].replace('# sent_id = ','').strip()
    if tmp_send_id in '0100_2000_6-doc1082.xml-4':
        tmp_send_id = '0100_2000_6-doc1082.xml-4a'
    tmp_err_list = [get_letters(i[:2]) for i in uni_conllu.split('<ns type="')][1:]
    
    for tmp_err in tmp_err_list:
        df_error_types.loc[tmp_send_id,tmp_err] = df_error_types.loc[tmp_send_id,tmp_err]
    

In [64]:
loc[idx, 'error_type']

'0100_2000_6-doc2664.xml-34'

'0102_2000_12-doc605.xml-7d'

In [80]:
'0100_2000_6-doc1082.xml-4a'.strip()

'0100_2000_6-doc1082.xml-4a'

In [90]:
df_error_types

,send_id,AS,UP,MP,FD,DV,WO,MJ,MD,MQ,...,UT,ID,MY,DQ,AG,TV,UQ,RQ,UJ,IA
unique_send_id,,,,,,,,,,,,,,,,,,,,,
0100_2000_6-doc2664.xml-34,0100_2000_6-doc2664.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0102_2000_6-doc648.xml-6,0102_2000_6-doc648.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0102_2000_6-doc1081.xml-16,0102_2000_6-doc1081.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0102_2000_12-doc724.xml-11,0102_2000_12-doc724.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc567.xml-13,0100_2000_12-doc567.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0102_2000_6-doc1098.xml-22,0102_2000_6-doc1098.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_6-doc166.xml-14,0100_2000_6-doc166.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc2263.xml-20,0100_2000_12-doc2263.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_6-doc395.xml-38,0100_2000_6-doc395.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
df_error_types.groupby(['send_id'])[err_train].sum()

,AS,UP,MP,FD,DV,WO,MJ,MD,MQ,L,...,UT,ID,MY,DQ,AG,TV,UQ,RQ,UJ,IA
send_id,,,,,,,,,,,,,,,,,,,,,
0100_2000_12-doc1002.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc1018.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc102.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc1029.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc1033.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc1035.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc1037.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc111.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100_2000_12-doc1225.xml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
